# Big Data házi feladat

Készítők: Csilling Tamás és Knyihár Gábor

## Feladatkiírás

Azonosító: BKK3

### Téma

Az I épületből este 10-kor, tömegközlekedéssel elérhető "kocsmák" (i.e., Google Maps találat erre a keresésre) elemzése

### Adatbeszerzés

https://openmobilitydata.org/p/bkk/42 és pl. Google Places API

### Adatelőkészítés és szűkítés

A Google Places API (a free változat elég kell, hogy legyen) talán a fő kihívás, egyenértékes megoldást elfogadok

### Leskálázás méretben "small" datára

Valószínűleg nem szükséges

### EDA fókusz

Az I épület 10 perces sétatávolságában megállóval rendelkező éjszakai járatok alapvető jellemzői (javasolt:
pandas_profiling vagy BambooLib)

### Big Data vizualizáció

Datashader alapú (lehet HoloViz-be integrálva) interaktív, Budapest-térkép alapú heatmap, mely az I épületből este 10-es
indulással, átszállás nélkül való utazási időben vett távolságát mutatja a "celláknak" (cellán belül átlagolással, ha
több opció is van). Természetesen vannak a városnak olyan részei, ahova átszállás nélkül nem lehet eljutni az
egyetemről!

### Elemzési feladat

Algoritmus azon megállók megtalálására, ahova 45 perc alatt, max k átszállással el lehet jutni az I épületből este 10-es
indulással. (Demonstráció elég k=1 vagy 2-re.)


## BKK adatok beszerzése

Az adatok elérhetőek a [https://www.bkk.hu/gtfs/budapest_gtfs.zip](https://www.bkk.hu/gtfs/budapest_gtfs.zip) linken, melyek [GTFS](https://developers.google.com/transit/gtfs/reference) formátumban vannak. Az elérhető adatok struktúráját mutatja a következő ábra:

![Chart](doc/assets/gtfs.png)

Ezek számos felesleges adatot is tartalmaznak, így ezt átalakítottuk és összefésültük egy nagy adathalmazba. Az oszlopok megfeleltetéseit mutatja a következő táblázat.

| Változó név                | Eredeti tábla | Eredeti név           |
|----------------------------|---------------|-----------------------|
| agency_name                | agency        | agency_name           |
| route_name                 | routes        | route_short_name      |
| route_type                 | routes        | route_type            |
| route_desc                 | routes        | route_desc            |
| trip_id                    | trips         | trip_id               |
| trip_direction             | trips         | direction_id          |
| trip_wheelchair_accessible | trips         | wheelchair_accessible |
| trip_bikes_allowed         | trips         | bikes_allowed         |
| trip_boarding_door         | trips         | boarding_door         |
| stop_arrival_time          | stop_times    | arrival_time          |
| stop_departure_time        | stop_times    | departure_time        |
| stop_id                    | stops         | stop_id               |
| stop_name                  | stops         | stop_name             |
| stop_latitude              | stops         | stop_lat              |
| stop_longitude             | stops         | stop_lon              |
| stop_location_type         | stops         | location_type         |
| stop_wheelchair_boarding   | stops         | wheelchair_boarding   |

Ezen felül néhány érték esetében alapértelmezetten `NaN` értékek kerültek be a táblázatba, ezeket át kellett alakítani, valamint kezelni kellett a időformátumokat is. A GTFS-ben az éjfél után közlekedő járatok hivatalosan még az adott naphoz tartoznak, ezért például egy 01:00-kor közlekedő járat ideje 25:00-ként van megjelölve. Ez viszont értelmezhetetlen a programkódban, így ezeket is át kellett alakítani. Viszont jelölni kellett, hogy az a következő napon van, azért, hogy könnyen lehessen kezelni az összehasonlításokat. A végeredményt mentettük parquett fájlba.

In [3]:
## ----- Imports -----

from datetime import datetime, timedelta
import os
import requests
import zipfile
import pandas as pd
import io

## ----- Functions  -----

def downloadFile(url, path, chunk_size=128):
    """Download zip file from url to path"""
    if os.path.isfile(path):
        return
    r = requests.get(url, stream=True)
    with open(path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

def getDataFrameFromZip(path, filename):
    """Extract pandas dataframe from zip file"""
    archive = zipfile.ZipFile(path, 'r')
    csv = archive.read(filename).decode("utf-8")
    handler = io.StringIO(csv)
    return pd.read_csv(handler,low_memory=False)

def convertTime(time):
    """Convert time string to time objects"""
    hours = int(time[0:2])
    if hours < 24:
        return datetime.strptime(time, "%H:%M:%S")
    else:
        new_time = str(hours-24).zfill(2) + time[2:]
        return datetime.strptime(new_time, "%H:%M:%S") + timedelta(days=1)

## ----- Variables -----

url = "https://www.bkk.hu/gtfs/budapest_gtfs.zip"
temp_folder = "temp"
zip_path = "temp/bkk.zip"
parquet_path = "temp/bkk.parquet.gzip"

## ----- Script -----

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

if not os.path.exists(parquet_path):
    # Download zip file
    downloadFile(url, zip_path)

    # Collect and clean data
    agency = getDataFrameFromZip(zip_path, "agency.txt")[["agency_id", "agency_name"]].set_index("agency_id")

    routes = getDataFrameFromZip(zip_path,"routes.txt")[["agency_id", "route_id", "route_short_name", "route_type", "route_desc"]].set_index(["route_id", "agency_id"])
    routes["route_type"] = routes["route_type"].astype(int)
    routes["route_type"] = routes["route_type"].replace([0,1,3,4,11,109],[0,1,2,3,4,5])
    routes = routes.rename(columns={'route_short_name': 'route_name'})

    trips = getDataFrameFromZip(zip_path,"trips.txt")[["route_id", "trip_id", "direction_id","wheelchair_accessible", "bikes_allowed", "boarding_door"]].set_index(["trip_id", "route_id"])
    trips = trips.fillna(0)
    trips["direction_id"] = trips["direction_id"].astype(int)
    trips["wheelchair_accessible"] = trips["wheelchair_accessible"].astype(int)
    trips["bikes_allowed"] = trips["bikes_allowed"].astype(int)
    trips["boarding_door"] = trips["boarding_door"].astype(int)
    trips["boarding_door"] = trips["boarding_door"].replace([0,2],[0,1])
    trips = trips.rename(columns={'direction_id': 'trip_direction', 'wheelchair_accessible': 'trip_wheelchair_accessible', 'bikes_allowed':'trip_bikes_allowed', 'boarding_door': 'trip_boarding_door'})

    stop_times = getDataFrameFromZip(zip_path,"stop_times.txt")[["stop_id", "trip_id", "arrival_time", "departure_time"]].set_index(["stop_id", "trip_id"])
    stop_times["departure_time"] = stop_times.departure_time.apply(convertTime)
    stop_times["arrival_time"] = stop_times.arrival_time.apply(convertTime)
    stop_times = stop_times.rename(columns={'arrival_time': 'stop_arrival_time', 'departure_time':'stop_departure_time'})

    stops = getDataFrameFromZip(zip_path,"stops.txt")[["stop_id", "stop_name", "stop_lat", "stop_lon", "location_type", "wheelchair_boarding"]].set_index("stop_id")
    stops = stops.fillna(0)
    stops["location_type"] = stops["location_type"].astype(int)
    stops["wheelchair_boarding"] = stops["wheelchair_boarding"].astype(int)
    stops = stops.rename(columns={'stop_lat': 'stop_latitude', 'stop_lon':'stop_longitude', 'location_type':'stop_location_type', 'wheelchair_boarding': 'stop_wheelchair_boarding'})

    # Join data and drop unnecessary columns
    all_data = trips.join(routes).join(stop_times).join(stops).join(agency)
    all_data = all_data.reset_index().drop(columns=["route_id", "agency_id"])

    # Order columns
    all_data = all_data[["agency_name","route_name","route_type","route_desc", "trip_id", "trip_direction",  "trip_wheelchair_accessible", "trip_bikes_allowed",  "trip_boarding_door",  "stop_arrival_time", "stop_departure_time",  "stop_id", "stop_name", "stop_latitude", "stop_longitude", "stop_location_type", "stop_wheelchair_boarding"]]

    # Export to parquet
    all_data.to_parquet(parquet_path, compression='gzip')

## BKK adatok értelmezése

Az adatokat parquet fájlból töltjük be, mely a következő oszlopokat tartalmazza:


| Változó                    | Változó neve               | Típus      | Megengedett értékek                                             | Leírás                                                                |
|----------------------------|----------------------------|------------|-----------------------------------------------------------------|-----------------------------------------------------------------------|
| Szolgáltató neve           | agency_name                | Szöveg     |                                                                 | A szolgáltató teljes neve.                                            |
| Járat neve                 | route_name                 | Szöveg     |                                                                 | Az járat rövid neve.                                                  |
| Járat típusa               | route_type                 | Egész szám | 0 - villamos, 1 - metró, 2 - busz, 3 - hajó, 4 - troli, 5 - hév | A járatot kiszolgáló jármű típusa.                                    |
| Járat leírása              | route_desc                 | Szöveg     |                                                                 | A járat rövid leírása.                                                |
| Útvonal azonosítója        | trip_id                    | Szöveg     |                                                                 | Két megálló közötti utazás azonosítója.                               |
| Útvonal iránya             | trip_direction             | Egész szám | 0 - normál, 1 - ellentétes                                      | Az utazás menetirányát jelzi.                                         |
| Kerekesszékkel elérhető    | trip_wheelchair_accessible | Egész szám | 0 - ismeretlen, 1 - igen, 2 - nem                               | Azt jelzi, hogy a járaton kerekesszékkel lehetséges-e utazni.         |
| Kerékpárok engedélyezettek | trip_bikes_allowed         | Egész szám | 0 - ismeretlen, 1 - igen, 2 - nem                               | Azt jelzi, hogy megengedett-e a kerékpár szállítás.                   |
| Beszálló ajtó              | trip_boarding_door         | Egész szám | 0 - bármelyik, 1 - első ajtó                                    | Azt jelzi, hogy melyik ajtón lehet-e felszállni.                      |
| Érkezési idő               | stop_arrival_time          | Idő        |                                                                 | Érkezési idő egy adott megállóhelyen egy adott utazáshoz.             |
| Indulási idő               | stop_departure_time        | Idő        |                                                                 | Indulási idő egy adott megállóból egy adott utazáshoz.                |
| Megálló azonosítója        | stop_id                    | Szöveg     |                                                                 | Megállóhelyet, állomást vagy állomás bejáratát azonosítja.            |
| Megálló neve               | stop_name                  | Szöveg     |                                                                 | A megálló neve.                                                       |
| Megálló helye (szélesség)  | stop_latitude              | Szám       | -90 - +90                                                       | A megálló koordinátájának szélességi foka.                            |
| Megálló helye (hosszúság)  | stop_longitude             | Szám       | -180 - +180                                                     | A megálló kordinátájának hosszúsági foka.                             |
| Megálló típusa             | stop_location_type         | Egész szám | 0 - megálló, 1 - állomás, 2 - állomás bejárat/kijárat           | A megálló típusa.                                                     |
| Kerekesszékes beszállás    | stop_wheelchair_boarding   | Egész szám | 0 - ismeretlen, 1 - igen, 2 - nem                               | Azt jelzi, hogy a megállóból lehetséges-e a kerekesszékes felszállás. |

In [4]:
df = pd.read_parquet(parquet_path)
display(df)

,agency_name,route_name,route_type,route_desc,trip_id,trip_direction,trip_wheelchair_accessible,trip_bikes_allowed,trip_boarding_door,stop_arrival_time,stop_departure_time,stop_id,stop_name,stop_latitude,stop_longitude,stop_location_type,stop_wheelchair_boarding
0,BKK,7G,2,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",B870931,0,1,2,0,1900-01-01 03:50:00,1900-01-01 03:50:00,008569,Cinkotai autóbuszgarázs,47.498051,19.236675,0,2
1,BKK,7G,2,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",B870931,0,1,2,0,1900-01-01 03:50:00,1900-01-01 03:50:00,F03291,Injekcióüzem,47.496206,19.231971,0,1
2,BKK,7G,2,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",B870931,0,1,2,0,1900-01-01 03:51:00,1900-01-01 03:51:00,F03403,EGIS Gyógyszergyár,47.497054,19.224595,0,1
3,BKK,7G,2,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",B870931,0,1,2,0,1900-01-01 03:52:00,1900-01-01 03:52:00,F03402,Zsemlékes út,47.503095,19.214434,0,1
4,BKK,7G,2,"Cinkotai autóbuszgarázs / Újpalota, Nyírpalota út",B870931,0,1,2,0,1900-01-01 03:53:00,1900-01-01 03:53:00,F03400,Petőfi tér,47.506473,19.211007,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723094,MÁV-HÉV,H5,5,Batthyány tér / Szentendre,H8000_22,0,0,0,0,1900-01-02 00:48:00,1900-01-02 00:48:00,F04692,"Budakalász, Lenfonó",47.621692,19.046912,0,0
5723095,MÁV-HÉV,H5,5,Batthyány tér / Szentendre,H8000_22,0,0,0,0,1900-01-02 00:50:00,1900-01-02 00:50:00,F04793,Szentistvántelep,47.629301,19.043159,0,0
5723096,MÁV-HÉV,H5,5,Batthyány tér / Szentendre,H8000_22,0,0,0,0,1900-01-02 00:52:00,1900-01-02 00:53:00,F04690,Pomáz,47.643188,19.032032,0,0
5723097,MÁV-HÉV,H5,5,Batthyány tér / Szentendre,H8000_22,0,0,0,0,1900-01-02 00:57:00,1900-01-02 00:57:00,F04688,Pannóniatelep,47.652488,19.065294,0,0
